Copyright (c) Facebook, Inc. and its affiliates.

This source code is licensed under the MIT license found in the
LICENSE file in the root directory of this source tree.

In [ ]:
import cv2
import numpy as np
from pyquaternion import Quaternion
import matplotlib.pyplot as plt
import scipy
import scipy.sparse
import scipy.sparse.linalg

# README

#### Dependencies: 
- OpenCV 2
- NumPy
- pyquaternion (`pip install pyquaternion`)

#### First, make sure it runs on the test sequence:
- Above, press `Cell > Run All`
- Make sure the 'output' folder is populated with frames, and the depth maps look reasonable
    
#### Then, you can input your own data:
- Replace the values of the three variables below
  - **input_frames**: the path to the folder which contains your video frames
  - **input_recon**: the path to the folder which contains your sparse reconstruction. For input format, we use the COLMAP format defined here (https://colmap.github.io/format.html). This folder should contain three files: `points2D.txt`, `images.txt`, and `cameras.txt`. Since the COLMAP format does not natively include information about whether or not a frame is a keyframe (since it's not originally intended for SLAM systems), we interpret this information from the POINTS2D[] in `images.txt`. That is, if a given image has a nonzero number of POINTS2D, we assume it is a keyframe.
  - **output_folder**: the path to the folder where you want the output saved

This Python notebook is intended as a reference implementation for research purposes. The code is an unoptimized version of the full method, and Python is slow, so performance will be worse than what is reported in the paper. Please refer to the paper for more accurate timing results.



## The parameters

In [ ]:
input_frames = "sample_data/frames/"
input_colmap = "sample_data/reconstruction/"
output_folder = "output/"

In [ ]:
# Algorithm parameters. See the paper for details.

tau_high = 0.9
tau_low = 0.1
tau_flow = 0.5
k_I = 5
k_T = 7
k_F = 31
lambda_d = 1
lambda_t = 0.01
lambda_s = 1


## A simplified COLMAP importer

In [ ]:
class Reconstruction:
    cameras = {}
    views = {}
    points3d = {}
    image_folder = ""
    
    def ViewIds(self):
        return list(self.views.keys())
    
    def GetNeighboringKeyframes(self, view_id):
        previous_keyframe = -1
        next_keyframe = -1
        for idx in range(view_id - 1, 0, -1):
            if self.views[idx].IsKeyframe():
                previous_keyframe = idx
                break
        for idx in range(view_id + 1, len(self.views) - 1):
            if self.views[idx].IsKeyframe():
                next_keyframe = idx
                break
        if previous_keyframe < 0 or next_keyframe < 0:
            return np.array()
        return np.array([previous_keyframe, next_keyframe])
    
    def GetReferenceFrames(self, view_id):
        kf = self.GetNeighboringKeyframes(view_id)
        dist = np.linalg.norm(self.views[kf[1]].Position() -\
                              self.views[kf[0]].Position()) / 2
        pos = self.views[view_id].Position()
        ref = []
        for idx in range(view_id + 1, len(self.views) - 1):
            if (np.linalg.norm(self.views[kf[1]].Position() -\
                              self.views[kf[0]].Position()) > dist):
                ref.append(idx)
                break
        for idx in range(view_id - 1, 0, -1):
            if (np.linalg.norm(self.views[kf[1]].Position() -\
                              self.views[kf[0]].Position()) > dist):
                ref.append(idx)
                break
        return ref

    def GetImage(self, view_id):
        return self.views[view_id].GetImage(self.image_folder)
    
    def GetSparseDepthMap(self, frame_id):
        camera = self.cameras[self.views[frame_id].camera_id]
        view = self.views[frame_id]
        view_pos = view.Position()
        depth_map = np.zeros((camera.width, camera.height), dtype=np.float32)
        for point_id, coord in view.points2d.items():
            pos3d = self.points3d[point_id].position3d
            depth = np.linalg.norm(pos3d - view_pos)
            depth_map[int(coord[0]), int(coord[1])] = depth
        return depth_map
    
    def Print(self):
        print "Found " + str(len(self.views)) + " cameras."
        for id in self.cameras:
            self.cameras[id].Print()
        print "Found " + str(len(self.views)) + " frames."
        for id in self.views:
            self.views[id].Print()

class Point:
    id = -1
    position3d = np.zeros(3, float)
            
class Camera:
    id = -1
    width = 0
    height = 0
    focal = np.zeros(2,float)
    principal = np.zeros(2,float)
    model = ""
    
    def Print(self):
        print "Camera " + str(self.id)
        print "-Image size: (" + str(self.width) + \
            ", " + str(self.height) + ")" 
        print "-Focal: " + str(self.focal) 
        print "-Model: " + self.model
        print ""

class View:
    id = -1
    orientation = Quaternion()
    translation = np.zeros(3, float)
    points2d = {}
    camera_id = -1
    name = ""
    
    def IsKeyframe(self):
        return len(self.points2d) > 0
    
    def Rotation(self):
        return self.orientation.rotation_matrix
    
    def Position(self):
        return self.orientation.rotate(self.translation)
    
    def GetImage(self, image_folder):
        mat = cv2.imread(image_folder + "/" + self.name)
        # Check that we loaded correctly.
        assert mat is not None, \
            "Image " + self.name + " was not found in " \
            + image_folder
        return mat
    
    def Print(self):
        print "Frame " + str(self.id) + ": " + self.name
        print "Rotation: \n" + \
            str(self.Rotation())
        print "Position: \n" + \
            str(self.Position())
        print ""
        
def ReadColmapCamera(filename):
    file = open(filename, "r")
    line = file.readline()
    cameras = {}
    while (line):
        if (line[0] != '#'):
            tokens = line.split()
            camera = Camera()
            camera.id = int(tokens[0])
            camera.model = tokens[1]
            # Currently we're assuming that the camera model
            # is in the SIMPLE_RADIAL format
            assert(camera.model == "PINHOLE")
            camera.width = int(tokens[2])
            camera.height = int(tokens[3])
            camera.focal[0] = float(tokens[4])
            camera.focal[1] = float(tokens[5])
            camera.principal[0] = float(tokens[6])
            camera.principal[1] = float(tokens[7])
            cameras[camera.id] = camera
        line = file.readline()
    return cameras;

def ReadColmapImages(filename):
    file = open(filename, "r")
    line = file.readline()
    views = {}
    while (line):
        if (line[0] != '#'):
            tokens = line.split()
            view = View()
            view.id = int(tokens[0])
            view.orientation.w = float(tokens[1])
            view.orientation.x = float(tokens[2])
            view.orientation.y = float(tokens[3])
            view.orientation.z = float(tokens[4])
            view.translation[0] = float(tokens[5])
            view.translation[1] = float(tokens[6])
            view.translation[2] = float(tokens[7])
            view.camera_id = int(tokens[8])
            view.name = tokens[9]
            line = file.readline()
            tokens = line.split()
            view.points2d = {}
            for idx in range(0, len(tokens) / 3):
                point_id = int(tokens[idx * 3 + 2])
                coord = np.array([float(tokens[idx * 3 + 0]), \
                         float(tokens[idx * 3 + 1])])
                view.points2d[point_id] = coord
            views[view.id] = view
            # Read the observations...
        line = file.readline()
    return views
           
def ReadColmapPoints(filename):
    file = open(filename, "r")
    line = file.readline()
    points = {}
    while (line):
        if (line[0] != '#'):
            tokens = line.split()
            point = Point()
            point.id = int(tokens[0])
            point.position3d = np.array([float(tokens[1]), \
                                        float(tokens[2]), \
                                        float(tokens[3])])
            points[point.id] = point
            
        line = file.readline()
    return points
        
            
    
def ReadColmap(poses_folder, images_folder):
    # Read the cameras (intrinsics)
    recon = Reconstruction()
    recon.image_folder = images_folder
    recon.cameras = ReadColmapCamera(poses_folder + "/cameras.txt")
    recon.views = ReadColmapImages(poses_folder + "/images.txt")
    recon.points3d = ReadColmapPoints(poses_folder + "/points3D.txt")
    print "Number of points: " + str(len(recon.points3d))
    print "Number of frames: " + str(len(recon.views))
    return recon

## The densification code

In [ ]:
def GetFlow(image1, image2):
    flow = cv2.calcOpticalFlowFarneback(\
        cv2.cvtColor(image1,cv2.COLOR_BGR2GRAY),\
        cv2.cvtColor(image2,cv2.COLOR_BGR2GRAY),\
        0.5, 3, 100, 100, 7, 1.5, 0)
    a,b,c = cv2.split(image1)
    return cv2.merge((a,b))

def GetFlowGradientMagnitude(flow):
    x1,x2 = cv2.split(cv2.Sobel(flow,cv2.CV_64F,1,0,ksize=5))
    y1,y2 = cv2.split(cv2.Sobel(flow,cv2.CV_64F,0,1,ksize=5))
    flow_grad_x = np.maximum(x1,x2)
    flow_grad_y = np.maximum(y1,y2)
    flow_grad_magnitude = cv2.sqrt((flow_grad_x * flow_grad_x) \
                                   + (flow_grad_y * flow_grad_y))
    reliability = np.zeros((flow.shape[0], flow.shape[1]))
    return flow_grad_magnitude, reliability

def GetImageGradientMagnitude(image):
    xr,xg,xb = cv2.split(cv2.Sobel(image,cv2.CV_64F,1,0,ksize=5))
    yr,yg,yb = cv2.split(cv2.Sobel(image,cv2.CV_64F,0,1,ksize=5))
    img_grad_x = np.maximum(xr,xg,xb)
    img_grad_y = np.maximum(yr,yg,yb)
    img_grad_magnitude = cv2.sqrt((img_grad_x * img_grad_x) \
                                  + (img_grad_y * img_grad_y))
    return img_grad_magnitude

def GetSoftEdges(image, flows):

    img_grad_magnitude = GetImageGradientMagnitude(image)
    flow_gradient_magnitude = np.zeros(img_gradient_magnitude.shape)
    max_reliability = np.zeros(img_gradient_magnitude.shape)
    for flow in flows:
        magnitude, reliability = GetFlowGradientMagnitude(flow)
        flow_gradient_magnitude[reliability > max_reliability] = magnitude
    
    flow_grad_magnitude = \
        cv2.GaussianBlur(flow_grad_magnitude,(k_F, k_F),0)
    return flow_grad_magnitude * (img_grad_magnitude)
    
def NonMaxSuppression(gradient):
    TG22 = 13573
    
    gx,gy = cv2.split(gradient * (2**15))
    mag = cv2.sqrt((gx * gx) \
                    + (gy * gy))
    suppressed = np.zeros(mag.shape)
    for x in range(1, gradient.shape[0] - 1):
        for y in range(1, gradient.shape[1] - 1):
            ax = int(abs(gx[x,y]))
            ay = int(abs(gy[x,y])) << 15
            tg22x = ax * TG22
            m = mag[x,y]
            if (ay < tg22x):
                if (m > mag[x,y-1] and\
                   m >= mag[x,y+1]):
                    suppressed[x,y] = m
            else:
                tg67x = tg22x + (ax << 16)
                if (ay > tg67x):
                    if (m > mag[x+1,y] and m >= mag[x-1,y]):
                        suppressed[x,y] = m
                else:
                    if (int(gx[x,y]) ^ int(gy[x,y]) < 0):
                        if (m > mag[x-1,y+1] and m >= mag[x+1,y-1]):
                            suppressed[x,y] = m
                    else:
                        if (m > mag[x-1,y-1] and m > mag[x+1,y+1]):
                            suppressed[x,y] = m
    return suppressed
                    
    
def Canny(soft_edges, image):
    image = cv2.GaussianBlur(image, (k_I, k_I), 0)
    xr,xg,xb = cv2.split(cv2.Sobel(image,cv2.CV_64F,1,0,ksize=5))
    yr,yg,yb = cv2.split(cv2.Sobel(image,cv2.CV_64F,0,1,ksize=5))
    image_gradient = cv2.merge((np.maximum(xr,xg,xb),np.maximum(yr,yg,yb)))
    print image_gradient.shape
    gradient_mag = NonMaxSuppression(image_gradient)
    return gradient_mag > 0
    
def DensifyFrame(sparse_points, hard_edges, soft_edges, last_depth_map):
    w = edges.shape[0]
    h = edges.shape[1]
    num_pixels = w * h
    A = scipy.sparse.dok_matrix((num_pixels * 3, num_pixels), dtype=np.float32)
    b = np.zeros(num_pixels, dtype=np.float32)
    num_entries = 0
    
    smoothness = np.maximum(1 - soft_edges, 0)
    
    for row in range(1,h - 1):
        for col in range(1,w - 1):
            # Add the data constraints
            if (sparse_points[col,row] > 0):
                A[num_entries, col + row * w] = lambda_d
                b[col + row * w] = sparse_points[col,row]
                num_entries += 1
            elif (last_depth_map.size > 0 and last_depth_map[col,row] > 0):
                A[num_entries, col + row * w] = lambda_t
                b[col + row * w] = last_depth_map[col,row]
                num_entries += 1
    
            # Add the smoothness constraints
            smoothness_weight = lambda_s * min(smoothness[col,row], \
                                               smoothness[col-1, row])
            if (int(hard_edges[col,row]) ^ int(hard_edges[col - 1, row]) == 0):
                A[num_entries, (col - 1) + row * w] = smoothness_weight
                A[num_entries, col + row * w] = -smoothness_weight
                num_entries += 1
            
            smoothness_weight = lambda_s * min(smoothness[col,row], \
                                               smoothness[col, row - 1])
            if (int(hard_edges[col,row]) ^ int(hard_edges[col, row - 1]) == 0):
                A[num_entries, col + (row - 1) * w] = smoothness_weight
                A[num_entries, col + (row - 1) * w] = -smoothness_weight
                num_entries += 1
    
    # Solve the system
    x = scipy.sparse.linalg.cg(A.transpose() * A, A.transpose() * b)
    depth = last_depth_map.clone()
    
    # Copy back the pixels
    for row in range(1,h):
        for col in range(1,w):
            depth[col,row] = x[col + row * w]
    return depth

def TemporalMedian(depth_maps):
    lists = {}
    depth_map = depth_maps.itervalues().next().clone()
    for row in range(0,h):
        for col in range(0,w):
            values = []
            for img in depth:
                if (img[col,row] > 0):
                    values.append(img[col, row])
            depth_map[col,row] = np.median(np.array(values))
    return depth_map
                

In [ ]:
recon = ReadColmap(input_colmap, input_frames)

last_depths = []
last_depth = np.array([])
for frame in recon.ViewIds():
    print "Processing frame " + recon.views[frame].name
    kfs = recon.GetReferenceFrames(frame)
    if (len(kfs) == 0):
        continue
    base_img = recon.GetImage(frame)
    flows = []
    for kf in kfs:
        img = recon.GetImage(kf) 
        flows.append(GetFlow(base_img, img))
    soft_edges = GetSoftEdges(img1, flows)
    edges = Canny(softedges, img1)
    depth = DensifyFrame(recon.GetSparseDepthMap(kf[0]), edges, \
                         soft_edges, last_depth)
    last_depths.append(depth)
    if (len(last_depths) > k_T):
        last_depths.pop(0)
    filtered_depth = TemporalFilter(last_depths)
    plt.imsave(output_folder + "/" + recon.views[frame].name, \
               filtered_depth)
    last_depth = depth